# Incomplete notebook for uncertainty estimation

In [ ]:
# same for uncertainty - 1 std
ERFstd = dict()
ERFstd["O3short"] = 9.9              # ozone shortwave
ERFstd["CH4"] = 6.9                  # methane
ERFstd["O3long"] = 3.4               # ozone longwave
ERFstd["SWV"] = 1.0                  # stratospheric water vapour
ERFstd["netNOx"] = 8.1               # nitrogen oxides (net)
ERFstd["BC"] = 165.5                 # black carbon
ERFstd["SO4"] = 16.0                 # sulphate
ERFstd["H2O"] = 0.0026               # water vapour
ERFstd["contrail"] = 0               # contrail and cirrus clouds [mW/m^2/km]

In [ ]:
# transient factor - what's that?
tf_years = np.arange(1980,2051)
transient_factor = np.empty(len(tf_years))

# spot data
transient_factor[tf_years == 2001] = 0.73
transient_factor[tf_years == 2005] = 0.75
transient_factor[tf_years == 2010] = 0.78
transient_factor[tf_years == 2018] = 0.79
transient_factor[tf_years == 2050] = 0.80

# keep constant into past & future
transient_factor[tf_years < 2001] = transient_factor[tf_years == 2001]

# interpolate in between
for p in [(2001,2005),(2005,2010),(2010,2018),(2018,2050)]:
    period = (tf_years >= p[0])*(tf_years <= p[1])
    slope = np.diff(transient_factor[period][[-1,0]])[0]/ \
                    np.diff(tf_years[period][[-1,0]])[0]
    transient_factor[period] = slope*(tf_years[period]-tf_years[period][0]) + \
                    transient_factor[period][0]

In [ ]:
# pull the data
NOx_xlsx = pd.read_excel(path+"data/NOx_radiative_forcing.xlsx")

# read out all values of O3short RF, CH4 RF, O3long RF and SWV RF from sheet
# use only those studies that have values for all 4, they are separated by 2 empty rows, find them
nox_allstudies = NOx_xlsx.values[:,ABC2index("Z")]
nox_allstudies = nox_allstudies[9:].astype(np.float)         # cut header
missing = -999999.0

NOxRFs = []  # all NOx forcings (o3short,ch4,o3long,stratospheric water vapour swv) for all studies
for i in np.arange(2,len(nox_allstudies)-1):
    
    # find a new set of studies (separated by 2 rows with nans)
    if ~np.isnan(nox_allstudies[i]) and np.isnan(nox_allstudies[i-1]) and np.isnan(nox_allstudies[i-2]):

        j = i+1
        
        # find the end of that set (separated by 2 rows with nans again)
        while ~(~np.isnan(nox_allstudies[j]) and 
                np.isnan(nox_allstudies[j-1]) and 
                np.isnan(nox_allstudies[j-2])) and j < len(nox_allstudies)-1:
            j+=1
        
        a = nox_allstudies[i:j-2]  # cut off last two nans
        if np.sum(~np.isnan(a)) % 4 == 0:  # only for 4i values in set ()
            a_reshaped = np.array(a[~np.isnan(a)]).reshape(4,-1).T  # reshape to represent individual studies
            for ar in a_reshaped:
                if not missing in ar:  # add only if missing is not in the study
                    NOxRFs.append(ar)

# convert to numpy array
NOxRFs = np.array(NOxRFs)

nstudies = NOxRFs.shape[0]                      # number of studies (60)
nox_years = np.arange(1980,2019)           # array with years
NOxA = np.empty((len(nox_years),nstudies)) # array of sum of NOx forcings by years and study 

for iy,year in enumerate(nox_years):
    transient_f = transient_factor[tf_years == year]
    for si in range(nstudies):
        o3short_rfs,ch4_rfs,o3long_rfs,swv_rfs = NOxRFs[si,:]
        NOxA[iy,si] = efficacy_o3short*o3short_rfs + \
                        efficacy_ch4*ch4_rfs*transient_f + \
                        efficacy_ch4*o3long_rfs*transient_f +\
                        efficacy_ch4*swv_rfs*transient_f

# Now finally the effective radiative forcing of NOx
noxm_rf = np.median(NOxA,axis=1)*nox
nox5_rf = np.percentile(NOxA,5,axis=1)*nox
nox95_rf = np.percentile(NOxA,95,axis=1)*nox